In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
from globalfunctions import *
from Indicators import *
import socket
from subprocess import check_output

url = 'https://www.invesco.com/us/financial-products/etfs/holdings/main/holdings/0?audienceType=Investor&action=download&ticker=QQQ'
#'https://www.invesco.com/us/financial-products/etfs/holdings/main/holdings/0?ticker=QQQ&action=download'
file_name = 'NASDAQ100.csv'


In [2]:
def isConnected():
    ipaddress = socket.gethostbyname(socket.gethostname())
    if ipaddress == '127.0.0.1':
        print('No internet, attempting to reconnect...')
        time.sleep(5)
        output = check_output('netsh wlan connect ssid="Kung Fu Kitty" name="Kung Fu Kitty"')
        time.sleep(15)
        return isConnected()
    else:
        print('Internet connected')
        return True



if isConnected():
    print('Success')
else:
    print('Failure')

Internet connected
Success


In [3]:
def checkTicker(ticker):
    ticker = ticker.replace(".","-")
    yahooURL = 'https://finance.yahoo.com/quote/{0}/'.format(ticker)
    #yahooURL = 'https://finance.yahoo.com/quote/{0}?p={1}'.format(t,t)
    xRes = requests.get(yahooURL,allow_redirects=False)
    return 200 == xRes.status_code
    

In [18]:
checkTicker('LGF.B')

True

In [3]:
if 1==0:
    #Gets the latest NASDAQ 100 ticker list
    #getCSV(url,file_name)

    #load NASDAQ100.csv into dataframe
    df = pd.read_csv(file_name, sep=',')

    dfInsert = df[['Fund Ticker','Security Identifier','Holding Ticker','Shares/Par Value','MarketValue','Weight','Name','Sector','Date']]

    #Update tblNASDAQ100 with latest data
    #Truncate Table tblNASDQ100; 
    iSQL = "Insert Into tblNASDQ100 (FundTicker,SecurityNum,HoldingsTicker,Shares,MarketValue,[Weight],companyName,Sector,CreateDate) VALUES(?,?,?,?,?,?,?,?,?)"
    dfTitles = ['Fund Ticker','Security Identifier','Holding Ticker','Shares/Par Value','MarketValue','Weight','Name','Sector','Date']
    insertTbl(df = dfInsert,iSQL = iSQL,dfTitles = dfTitles)
    print('Complete')

In [4]:
# Get all tickers from SQL DB
df = getData("Select Distinct ticker as 'HoldingsTicker', 'fkStrategyID' = SUBSTRING(fkStrategyID,0,len(fkStrategyID)) From vw_tblStocks")
originalTickerCount = df.shape[0]
print("{0} tickers".format(originalTickerCount))


401 tickers


In [5]:
#Generate list of tickers with strategy to collect data
tickers = []
for dr in df.iterrows():
    t = dr[1]['HoldingsTicker'].strip()
    fkStrategyID = dr[1]['fkStrategyID'].split(';')
    tickers.append({'ticker': t.strip(), 'fkStrategyID': fkStrategyID, 'isUpdated':0})


print("{0} tickers in list".format(len(tickers)))

401 tickers in list


In [10]:
def getMomentumData(t,td_ameritrade_key):
    ticker = t['ticker']    
    #Use TD Ameritrade API to get price history for ticker
    priceHistory = get_price_history(td_ameritrade_key,symbol=ticker, period=6, periodType='month', frequencyType='daily')
    
    i = len(priceHistory['candles'])-1
    j = i-100
    k = j-10
    #Calculate 20 week ROC (HL/2)
    if i>110:
        numerator = ((priceHistory['candles'][i]['high'] + priceHistory['candles'][i]['low']))/2
        denominator = ((priceHistory['candles'][j]['high'] + priceHistory['candles'][j]['low'])+(priceHistory['candles'][k]['high'] + priceHistory['candles'][k]['low']))/4
        roc = numerator/denominator
        hv = historicalVolatility(priceHistory,101,'dd')
    else:
        roc = -1.0
        hv = 0.0
    
    #Make GET Request to get 52-week high and calculate percent from high based on latest close
    r = get_fundemental_data(td_ameritrade_key,ticker)
    #Navigate returned dictionary object 
    yearHigh = float(r[ticker]['fundamental']['high52'])
    
    lastCandle = priceHistory['candles'][i]
    avgClosePrice = (lastCandle['high'] + lastCandle['low'] + lastCandle['close'])/3
    closePrice = lastCandle['close']
    percentFromHigh = avgClosePrice/yearHigh

    #append scrapped data into data list to be stored in db later
    d = []
    for sID in t['fkStrategyID']:
        d.append({'Ticker':ticker,'closePrice':closePrice,'yearHigh':yearHigh,'percentFromHigh':percentFromHigh,'sixMonthReturn':roc,'historicalVolatility':hv,'fkPortfolioStrategyID':sID})
    time.sleep(1)
    return d


In [7]:
d = []
key =  getData("SELECT [keyValue] FROM [fattailinvestorweb].[dbo].[tblAPIKeys] where pkAPIID = 1")
td_ameritrade_key = key['keyValue'][0]


In [21]:
priceHistory = get_price_history(td_ameritrade_key,symbol='AAPL', period=6, periodType='month', frequencyType='daily')
print(priceHistory['candles'][125])

{'open': 143.07, 'high': 145.09, 'low': 136.54, 'close': 142.92, 'volume': 157611710, 'datetime': 1611554400000}


## Removing Tickers as needed
> This code is to manually remove a ticker from the ticker list, need to then follow up and remove from DB to prevent issues in the future. This is required because stocks get delisted, change their ticker, etc.

In [12]:
i = 0
ticker = 'IPOC'
for index, item in enumerate(tickers):
    if item['ticker'] == ticker:
        i = index      
        

        


In [13]:
print(tickers[i]['ticker'])

IPOC


In [14]:
tickers[i]['ticker'] = 'CLOV'
print(tickers[i]['ticker'])

CLOV


## Get the data

In [16]:
# This will scrape data AND call the TD Ameritrade API
cntRemaining = [n for n in tickers if n['isUpdated'] == 0]
cntRemaining = len(cntRemaining)+1
print('Remaining: ' + str(cntRemaining))
for t in tickers:
    if t['isUpdated'] == 0:
        try:
            print("Count Remaining {0}".format(cntRemaining))
            cntRemaining -= 1
            if isConnected():
                print('Collecting ' + t['ticker'])
                for n in getMomentumData(t,td_ameritrade_key):
                    d.append(n)
                t['isUpdated'] = 1
                time.sleep(0.25)
        except:
            try:
                time.sleep(1)
                if isConnected():
                    print("** SECOND TRY FOR " + t['ticker']) + " ***"                            
                    for n in getMomentumData(t,td_ameritrade_key):
                        d.append(n)
                    t['isUpdated'] = 1
            except:
                print('*** FAILED FOR ' + t['ticker']) + " ***"
                pass
print('Done Collecting')

Remaining: 119
Count Remaining 119
Internet connected
Count Remaining 118
Internet connected
Count Remaining 117
Internet connected
Count Remaining 116
Internet connected
Count Remaining 115
Internet connected
Count Remaining 114
Internet connected
Count Remaining 113
Internet connected
Count Remaining 112
Internet connected
Count Remaining 111
Internet connected
Count Remaining 110
Internet connected
Count Remaining 109
Internet connected
Count Remaining 108
Internet connected
Count Remaining 107
Internet connected
Count Remaining 106
Internet connected
Count Remaining 105
Internet connected
Count Remaining 104
Internet connected
Count Remaining 103
Internet connected
Count Remaining 102
Internet connected
Count Remaining 101
Internet connected
Count Remaining 100
Internet connected
Count Remaining 99
Internet connected
Count Remaining 98
Internet connected
Count Remaining 97
Internet connected
Count Remaining 96
Internet connected
Count Remaining 95
Internet connected
Count Remaining

In [17]:
t = [i['Ticker'] for i in d]
missingTickers = [n['ticker'] for n in tickers if n['ticker'] not in t]
assert len(missingTickers) ==0

In [19]:
d[348]

{'Ticker': 'TRV',
 'closePrice': 146.36,
 'yearHigh': 152.293,
 'percentFromHigh': 0.9546947878978899,
 'sixMonthReturn': 1.2469935245143386,
 'historicalVolatility': 0.28297793910729635,
 'fkPortfolioStrategyID': '1005'}

In [20]:
#convert data list to dataFrame
headers = ['Ticker','closePrice','yearHigh','percentFromHigh','sixMonthReturn','historicalVolatility','fkPortfolioStrategyID']
df2 = pd.DataFrame(d,columns=headers)
iSQL = 'INSERT INTO tblStockPerformance (ticker,closePrice,yearHigh,percentFromHigh,sixMonthReturn,historicalVolatility,fkPortfolioStrategyID) Values(?,?,?,?,?,?,?)'

#store dataFrame into sql db
insertTbl(df2,iSQL,headers)

In [31]:
#get top 10 positions with relative momentum filter inplace
#ie Top 10 performing assets, then drop those under performing the 
#worst performing SAFEHAVEN asset. Should be 10 or less tickers
#sSQL = ";with cte_one as( Select Top 10 ticker, percentFromHigh, 'shares' = cast(round(?/closePrice,2) as decimal(5,2)), closePrice, cast(createDate as date) as 'createDate' From tblStockPerformance Where cast(createDate as date) = (Select Max(cast(createDate as date)) From tblStockPerformance) Order by percentFromHigh desc) Select ticker, closePrice From cte_one where percentFromHigh >= (Select min(percentFromHigh) From cte_one where ticker in ('IEF','TLT','BND','SHY','GLD')) "
sSQL = "Select Top 10 ticker, closePrice From vw_tblStockPerformance Where fkPortfolioStrategyID = 1 AND momentum >=(Select min(momentum) From vw_tblStockPerformance Where ticker in ('SHY','IEF','BND','GLD','TLT')) Order by momentum desc"
df = getData(sSQL)

print(df)


  ticker  closePrice
0   TSLA    485.7900
1     ZM    317.1000
2   NVDA    541.2100
3   PYPL    204.0300
4   AAPL    130.6400
5   LULU    378.0500
6   MELI   1156.6851
7   CTAS    332.4350
8    AMD     92.0200
9     JD     78.9350


In [100]:
#establish portfolio size and position count
portfolioSize = 10000.00
nPositions = 10

#Equally divide funds into ideal portfolio size
positionSize = portfolioSize/nPositions
#initalize empty position list to be filled based on query results
positions = []

#for each ticker in the portfolio, add funds for position
for r in df.iterrows():
    ticker = r[1]['ticker']
    ammount = positionSize
    portfolioSize -= positionSize
    closePrice = r[1]['closePrice']
    positions.append({'ticker':ticker,'ammount':ammount,'closePrice':closePrice,'shareCount':0.0})

#if funds remain, split remaning equally into SAFEHAVEN assets
if portfolioSize >0:
    filterSafeHavens = filter(lambda x:x['ticker'] in SAFEHAVENS,positions)
    cntSafeHavens = len(list(filterSafeHavens))
    extraFunds = portfolioSize/cntSafeHavens
    for p in (p for p in positions if p['ticker'] in SAFEHAVENS):
        p['ammount'] += extraFunds

#With dollar ammount allocated, calculate estimate share count
for p in positions:
    p['shareCount'] = round(p['ammount']/p['closePrice'],2)

#convert position list to dataFrame
dfOrders = pd.DataFrame(positions,columns = ['ticker','ammount','closePrice','shareCount'])
print(dfOrders)
    


  ticker  ammount  closePrice  shareCount
0  DXCM    1000.0      341.52        2.93
1    IEF   1000.0      121.71        8.22
2    SHY   1000.0       86.60       11.55
3    GLD   1000.0      159.78        6.26
4    ZM    1000.0      138.56        7.22
5  REGN    1000.0      525.50        1.90
6  PYPL    1000.0      120.61        8.29
7  CTXS    1000.0      142.58        7.01
8  NFLX    1000.0      415.27        2.41
9  AMZN    1000.0     2286.04        0.44


In [117]:
for t in tickers:
    print(t)

{'ticker': 'AAPL', 'fkStrategyID': ['1', '1005'], 'isUpdated': 1}
{'ticker': 'ADBE', 'fkStrategyID': ['1'], 'isUpdated': 1}
{'ticker': 'ADI', 'fkStrategyID': ['1'], 'isUpdated': 1}
{'ticker': 'ADP', 'fkStrategyID': ['1'], 'isUpdated': 1}
{'ticker': 'ADSK', 'fkStrategyID': ['1'], 'isUpdated': 1}
{'ticker': 'ALGN', 'fkStrategyID': ['1'], 'isUpdated': 1}
{'ticker': 'ALXN', 'fkStrategyID': ['1'], 'isUpdated': 1}
{'ticker': 'AMAT', 'fkStrategyID': ['1'], 'isUpdated': 1}
{'ticker': 'AMD', 'fkStrategyID': ['1'], 'isUpdated': 1}
{'ticker': 'AMGN', 'fkStrategyID': ['1'], 'isUpdated': 1}
{'ticker': 'AMZN', 'fkStrategyID': ['1'], 'isUpdated': 1}
{'ticker': 'ANSS', 'fkStrategyID': ['1'], 'isUpdated': 1}
{'ticker': 'ASML', 'fkStrategyID': ['1'], 'isUpdated': 1}
{'ticker': 'ATVI', 'fkStrategyID': ['1'], 'isUpdated': 1}
{'ticker': 'AVGO', 'fkStrategyID': ['1'], 'isUpdated': 1}
{'ticker': 'AXP', 'fkStrategyID': ['1005'], 'isUpdated': 1}
{'ticker': 'BA', 'fkStrategyID': ['1005'], 'isUpdated': 1}
{'tick